# TODO: Title

This notebook lists all the steps that you need to complete the complete this project. You will need to complete all the TODOs in this notebook as well as in the README and the two python scripts included with the starter code.


**TODO**: Give a helpful introduction to what this notebook is for. Remember that comments, explanations and good documentation make your project informative and professional.

**Note:** This notebook has a bunch of code and markdown cells with TODOs that you have to complete. These are meant to be helpful guidelines for you to finish your project while meeting the requirements in the project rubrics. Feel free to change the order of these the TODO's and use more than one TODO code cell to do all your tasks.

In [1]:
# TODO: Install any packages that you might need
# For instance, you will need the smdebug package
!pip install smdebug

In [35]:
# TODO: Import any packages that you might need
# For instance you will need Boto3 and Sagemaker
import sagemaker
import boto3, os, sys

In [27]:
from sagemaker import get_execution_role
role = get_execution_role()
sess = sagemaker.Session()
sess.default_bucket(), sess.boto_region_name, get_execution_role()

('sagemaker-us-east-1-176302953206',
 'us-east-1',
 'arn:aws:iam::176302953206:role/service-role/AmazonSageMaker-ExecutionRole-20211110T114740')

In [4]:
bucket= 'image-classification-using-aws-sagemaker' 
bucket

'image-classification-using-aws-sagemaker'

In [5]:
os.environ["DEFAULT_S3_BUCKET"] = bucket
! echo "$DEFAULT_S3_BUCKET"

image-classification-using-aws-sagemaker


## Dataset
TODO: Explain what dataset you are using for this project. Maybe even give a small overview of the classes, class distributions etc that can help anyone not familiar with the dataset get a better understand of it.

In [ ]:
#TODO: Fetch and upload the data to AWS S3
# Command to download and unzip data
#!wget -O https://s3-us-west-1.amazonaws.com/udacity-aind/dog-project/dogImages.zip
    
!wget -q https://s3-us-west-1.amazonaws.com/udacity-aind/dog-project/dogImages.zip -O ./dogImages.zip

!unzip dogImages.zip

In [8]:
! echo s3://${DEFAULT_S3_BUCKET}/valid/

s3://image-classification-using-aws-sagemaker/valid/


In [ ]:
!aws s3 sync ./dogImages/valid  s3://${DEFAULT_S3_BUCKET}/valid/

In [ ]:
!aws s3 sync ./dogImages/test  s3://${DEFAULT_S3_BUCKET}/test/

In [ ]:
!aws s3 sync ./dogImages/train s3://${DEFAULT_S3_BUCKET}/train/

In [ ]:
bucket_petit_jeu_de_donnees = 'cifar10-little-dataset' 
from torchvision.datasets import CIFAR10
from torchvision import transforms


local_dir = 'data'
CIFAR10.mirrors = ["https://sagemaker-sample-files.s3.amazonaws.com/datasets/image/CIFAR10/"]
CIFAR10(
    local_dir,
    download=True,
    transform=transforms.Compose(
        [transforms.ToTensor()]
    )
)

sagemaker_session = sagemaker.Session()
inputs_little_dataset = sagemaker_session.upload_data(path="data", bucket=bucket_petit_jeu_de_donnees, key_prefix="")
print("input spec (in this case, just an S3 path): {}".format(inputs_little_dataset))

## Hyperparameter Tuning
**TODO:** This is the part where you will finetune a pretrained model with hyperparameter tuning. Remember that you have to tune a minimum of two hyperparameters. However you are encouraged to tune more. You are also encouraged to explain why you chose to tune those particular hyperparameters and the ranges.

**Note:** You will need to use the `hpo.py` script to perform hyperparameter tuning.

In [12]:
!pip install torch

In [13]:
!pip install torchvision

In [15]:
import hpo

In [17]:
from sagemaker.tuner import (
    IntegerParameter,
    CategoricalParameter,
    ContinuousParameter,
    HyperparameterTuner,
)

In [24]:
#TODO: Declare your HP ranges, metrics etc.
hyperparameter_ranges = {
    "batch_size" : CategoricalParameter([32, 64, 128]),
    "learning_rate" : ContinuousParameter(0.001, 0.01),
    #"momentum" : ContinuousParameter(0.2, 0.3)    
}

objective_metric_name = "average test loss"
objective_type = "Minimize"
metric_definitions = [{"Name": "average test loss", "Regex": "Test set: Average loss: ([0-9\\.]+)"}]

In [40]:
#TODO: Create estimators for your HPs

# TODO: Your estimator here
estimator = PyTorch(
    entry_point="hpo.py",
    role=role,
    py_version='py36',
    framework_version="1.8",
    instance_count=1,
    #instance_type="ml.m5.large"
    instance_type="ml.p3.2xlarge",
)

# TODO: Your HP tuner here
tuner = HyperparameterTuner(
    estimator,
    objective_metric_name,
    hyperparameter_ranges,
    metric_definitions,
    max_jobs=1,
    max_parallel_jobs=1,
    objective_type=objective_type)

In [41]:
{"training": f"s3://{bucket}/"}

{'training': 's3://image-classification-using-aws-sagemaker/'}

In [ ]:
%%timeit
# TODO: Fit your HP Tuner
tuner.fit({"training": f"s3://{bucket}/"}, wait=True) # TODO: Remember to include your data channels

............................

In [ ]:
# TODO: Get the best estimators and the best HPs

best_estimator = tuner.best_estimator()#TODO

#Get the hyperparameters of the best trained model
#hyperparameters = best_estimator.hyperparameters()
#hyperparameters



hyperparameters = {"batch_size": int(best_estimator.hyperparameters()['batch_size'].replace('"', '')), \
                   "learning_rate": best_estimator.hyperparameters()['learning_rate']}
hyperparameters

## Model Profiling and Debugging
TODO: Using the best hyperparameters, create and finetune a new model

**Note:** You will need to use the `train_model.py` script to perform model profiling and debugging.

In [21]:
from sagemaker.pytorch import PyTorch
from sagemaker import get_execution_role
from sagemaker.debugger import (
    Rule,
    DebuggerHookConfig,
    rule_configs,
)

In [22]:
import train_model

In [ ]:
# TODO: Set up debugging and profiling rules and hooks
rules = [
    Rule.sagemaker(rule_configs.loss_not_decreasing()),
    Rule.sagemaker(rule_configs.vanishing_gradient()),
    Rule.sagemaker(rule_configs.overfit()),
    Rule.sagemaker(rule_configs.overtraining()),
    Rule.sagemaker(rule_configs.poor_weight_initialization()),
    ProfilerRule.sagemaker(rule_configs.ProfilerReport()),
]


hook_config = DebuggerHookConfig(
    hook_parameters={
        "train.save_interval": "100",
        "eval.save_interval": "10"
    }
)

profiler_config = ProfilerConfig(
    system_monitor_interval_millis=500, framework_profile_params=FrameworkProfile(num_steps=2)
)

debugger_config = DebuggerHookConfig(hook_parameters={"train.save_interval":"100", "eval.save_interval":"10"})

In [ ]:
# TODO: Create and fit an estimator

# TODO: Your estimator here
estimator = PyTorch(
    role=get_execution_role(),
    instance_count=1,
    instance_type="ml.p3.2xlarge",
    entry_point="train_model.py",
    framework_version="1.8",
    py_version="py36",
    hyperparameters=hyperparameters,
    profiler_config=profiler_config,
    debugger_hook_config=debugger_config,
    rules=rules,
)

estimator.fit({"training": f"s3://{bucket}/"}, wait=True)

In [ ]:
# TODO: Plot a debugging output.

from smdebug.trials import create_trial
from smdebug.core.modes import ModeKeys

trial = create_trial(estimator.latest_job_debugger_artifacts_path())


In [ ]:
trial.tensor_names()

In [ ]:
len(trial.tensor("nll_loss_output_0").steps(mode=ModeKeys.TRAIN))

In [ ]:
len(trial.tensor("nll_loss_output_0").steps(mode=ModeKeys.EVAL))

In [ ]:
def get_data(trial, tname, mode):
    tensor = trial.tensor(tname)
    steps = tensor.steps(mode=mode)
    vals = []
    for s in steps:
        vals.append(tensor.value(s, mode=mode))
    return steps, vals



In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import host_subplot


def plot_tensor(trial, tensor_name):

    steps_train, vals_train = get_data(trial, tensor_name, mode=ModeKeys.TRAIN)
    print("loaded TRAIN data")
    steps_eval, vals_eval = get_data(trial, tensor_name, mode=ModeKeys.EVAL)
    print("loaded EVAL data")

    fig = plt.figure(figsize=(10, 7))
    host = host_subplot(111)

    par = host.twiny()

    host.set_xlabel("Steps (TRAIN)")
    par.set_xlabel("Steps (EVAL)")
    host.set_ylabel(tensor_name)

    (p1,) = host.plot(steps_train, vals_train, label=tensor_name)
    print("completed TRAIN plot")
    (p2,) = par.plot(steps_eval, vals_eval, label="val_" + tensor_name)
    print("completed EVAL plot")
    leg = plt.legend()

    host.xaxis.get_label().set_color(p1.get_color())
    leg.texts[0].set_color(p1.get_color())

    par.xaxis.get_label().set_color(p2.get_color())
    leg.texts[1].set_color(p2.get_color())

    plt.ylabel(tensor_name)

    plt.show()

In [ ]:
plot_tensor(trial, "nll_loss_output_0")

**TODO**: Is there some anomalous behaviour in your debugging output? If so, what is the error and how will you fix it?  
**TODO**: If not, suppose there was an error. What would that error look like and how would you have fixed it?

In [ ]:
# TODO: Display the profiler output
rule_output_path = estimator.output_path + estimator.latest_training_job.job_name + "/rule-output"

# get the autogenerated folder name of profiler report
profiler_report_name = [
    rule["RuleConfigurationName"]
    for rule in estimator.latest_training_job.rule_job_summary()
    if "Profiler" in rule["RuleConfigurationName"]
][0]

import IPython

IPython.display.HTML(filename=profiler_report_name + "/profiler-output/profiler-report.html")

## Model Deploying

In [ ]:
# TODO: Deploy your model to an endpoint

# TODO: Add your deployment configuration like instance type and number of instances
predictor=estimator.deploy(initial_instance_count=1, instance_type='ml.t3.medium') 


In [ ]:
# TODO: Run an prediction on the endpoint

import requests
request_dict={ "url": "https://s3.amazonaws.com/cdn-origin-etr.akc.org/wp-content/uploads/2017/11/20113314/Carolina-Dog-standing-outdoors.jpg" }
image = requests.get(request_dict['url']).content

image = # TODO: Your code to load and preprocess image to send to endpoint for prediction
response = predictor.predict(image, initial_args={"ContentType": "image/jpeg"})
response

In [ ]:
# TODO: Remember to shutdown/delete your endpoint once your work is done
predictor.delete_endpoint()